<a href="https://colab.research.google.com/github/ayberkcorp/cap5610-machine_learning/blob/master/hw5/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Data Handling

In [1]:
import time
from math import *
from tkinter import *
import random
import pandas as pd
from google.colab import files

team_data = [["X1", 3, 5],
             ["X2", 3, 4],
             ["X3", 2, 8],
             ["X4", 2, 3],
             ["X5", 6, 2],
             ["X6", 6, 4],
             ["X7", 7, 3],
             ["X8", 7, 4],
             ["X9", 8, 5],
             ["X10", 7, 6]]

uploaded = files.upload()

with open('iris.data', 'rt') as file1:
  FileasList = file1.readlines()
iris_data = []


def is_number_string(str):
  if len(str) == 0:
    return False
  if  len(str) > 1 and str[0] == "-":
      str = s[1:]
  for c in str:
    if c not in "0123456789.":
      return False
  return True

for line in FileasList:
  line_list = (f'x{len(iris_data)},'+line).strip().replace('"', '').split(",")
  for i in range(len(line_list)):
    if (is_number_string(line_list[i])):
      line_list[i] = float(line_list[i])
  instance = tuple(line_list)
  iris_data.append(instance)

def is_number_string(str):
  if len(str) == 0:
    return False
  if  len(str) > 1 and str[0] == "-":
      str = s[1:]
  for c in str:
    if c not in "0123456789.":
      return False
  return True

Saving iris.data to iris (7).data


# Functions for k-means clustering

In [2]:
def kmeans(instances, k, metric, iters=-1, initCentroids=None):
    result = {}
    if (initCentroids == None or len(initCentroids) < k):
        random.seed(time.time())
        centroids = random.sample(instances, k)
    else:
        centroids = initCentroids
    prevCentroids = []
    iteration = 0
    prevwith = 1e70
    while (centroids != prevCentroids or iters < -2):
        iteration += 1
        clusters = assignAll(instances, centroids, metric)
        prevCentroids = centroids
        centroids = computeCentroids(clusters)
        withinss = computeWithinss(clusters, centroids, metric)
        if(iteration == iters or (iters < -2 and iteration == -iters)):
            break
        if(iters == -2 and withinss > prevwith):
            break
        prevwith = withinss
    result["clusters"] = clusters
    result["centroids"] = centroids
    result["withinss"] = withinss
    result["iterations"] = iteration
    return result

def distance(instance1, instance2, metric):
    if instance1 == None or instance2 == None:
        return float("inf")
    dist = 0
    a, b = 0, 0
    for i in range(1, len(instance1)):
        if(isinstance(instance1[i], str) or isinstance(instance2[i], str)):
            continue
        if(metric == 'euclidean'):
            dist += (instance1[i] - instance2[i])**2
        elif(metric == 'manhattan'):
            dist += abs(instance1[i] - instance2[i])
        elif(metric == 'jaccard'):
            a += min(instance1[i], instance2[i])
            b += max(instance1[i], instance2[i])
        elif(metric == 'cosine'):
            dist += instance1[i]*instance2[i]
            a += instance1[i]*instance1[i]
            b += instance2[i]*instance2[i]
    if(metric == 'jaccard'):
        dist = 1 - a / b
    elif(metric == 'cosine'):
        dist = 1 - dist / sqrt(a * b)
    return dist

def assign(instance, centroids, metric):
    minDistance = distance(instance, centroids[0], metric)
    minDistanceIndex = 0
    for i in range(1, len(centroids)):
        d = distance(instance, centroids[i], metric)
        if (d < minDistance):
            minDistance = d
            minDistanceIndex = i
    return minDistanceIndex

def meanInstance(name, instanceList):
    numInstances = len(instanceList)
    if (numInstances == 0):
        return
    numAttributes = len(instanceList[0])
    abc = {}
    means = [name] + [0] * (numAttributes - 1)
    for instance in instanceList:
        for i in range(1, numAttributes):
            if(isinstance(instance[i], str)):
                if(not instance[i] in abc):
                    abc[instance[i]] = 0
                abc[instance[i]] += 1
                means[i] = MaxKey = max(abc, key=abc.get)
                continue
            means[i] += instance[i]
    for i in range(1, numAttributes):
        if(isinstance(instance[i], str)):
            continue
        means[i] /= float(numInstances)
    return tuple(means)

def assignAll(instances, centroids, metric):
    clusters = createEmptyListOfLists(len(centroids))
    for instance in instances:
        clusterIndex = assign(instance, centroids, metric)
        clusters[clusterIndex].append(instance)
    return clusters

def computeCentroids(clusters):
    centroids = []
    for i in range(len(clusters)):
        name = "centroid" + str(i)
        centroid = meanInstance(name, clusters[i])
        centroids.append(centroid)
    return centroids

def computeWithinss(clusters, centroids, metric):
    result = 0
    for i in range(len(centroids)):
        centroid = centroids[i]
        cluster = clusters[i]
        for instance in cluster:
            result += distance(centroid, instance, metric) ** 2
    return result

def createEmptyListOfLists(numSubLists):
    myList = []
    for i in range(numSubLists):
        myList.append([])
    return myList

# Task 1

### Q1:

In [3]:
k_means_data = kmeans(team_data, 2, 'manhattan', 1, [['C1', 4, 6], ['C2', 5, 4]])
print(k_means_data)

{'clusters': [[['X1', 3, 5], ['X3', 2, 8], ['X10', 7, 6]], [['X2', 3, 4], ['X4', 2, 3], ['X5', 6, 2], ['X6', 6, 4], ['X7', 7, 3], ['X8', 7, 4], ['X9', 8, 5]]], 'centroids': [('centroid0', 4.0, 6.333333333333333), ('centroid1', 5.571428571428571, 3.5714285714285716)], 'withinss': 83.22448979591836, 'iterations': 1}


### Q2:

In [4]:
k_means_data = kmeans(team_data, 2, 'euclidean', 1, [['C1', 4, 6], ['C2', 5, 4]])
print(k_means_data)

{'clusters': [[['X1', 3, 5], ['X3', 2, 8]], [['X2', 3, 4], ['X4', 2, 3], ['X5', 6, 2], ['X6', 6, 4], ['X7', 7, 3], ['X8', 7, 4], ['X9', 8, 5], ['X10', 7, 6]]], 'centroids': [('centroid0', 2.5, 6.5), ('centroid1', 5.75, 3.875)], 'withinss': 387.509765625, 'iterations': 1}


### Q3:

In [5]:
k_means_data = kmeans(team_data, 2, 'manhattan', 1, [['C1', 3, 3], ['C2', 8, 3]])
print(k_means_data)

{'clusters': [[['X1', 3, 5], ['X2', 3, 4], ['X3', 2, 8], ['X4', 2, 3]], [['X5', 6, 2], ['X6', 6, 4], ['X7', 7, 3], ['X8', 7, 4], ['X9', 8, 5], ['X10', 7, 6]]], 'centroids': [('centroid0', 2.5, 5.0), ('centroid1', 6.833333333333333, 4.0)], 'withinss': 40.5, 'iterations': 1}


### Q4:

In [6]:
k_means_data = kmeans(team_data, 2, 'manhattan', 1, [['C1', 3, 2], ['C2', 4, 8]])
print(k_means_data)

{'clusters': [[['X1', 3, 5], ['X2', 3, 4], ['X4', 2, 3], ['X5', 6, 2], ['X6', 6, 4], ['X7', 7, 3], ['X8', 7, 4]], [['X3', 2, 8], ['X9', 8, 5], ['X10', 7, 6]]], 'centroids': [('centroid0', 4.857142857142857, 3.5714285714285716), ('centroid1', 5.666666666666667, 6.333333333333333)], 'withinss': 96.25850340136054, 'iterations': 1}


# Task 2

In [11]:
euclidean = kmeans(iris_data, 3, 'euclidean')
cosine = kmeans(iris_data, 3, 'cosine')
jaccard = kmeans(iris_data, 3, 'jaccard')
print(euclid['withinss'], cosine['withinss'], jaccard['withinss'])

IndexError: ignored